In [21]:
import pandas as pd
import numpy as np
import folium
from IPython.display import display

np.random.seed(42)

# Data obtained from: https://people.sc.fsu.edu/~jburkardt/datasets/cities/cities.html
url = 'https://raw.githubusercontent.com/grantaguinaldo/ssie/master/ssie641/assignment_1_data.csv'
df = pd.read_csv(url)
df['idx'] = df.index + 1

df_dict = df.to_dict(orient = 'records')

def distance(x_test = None, y_test = None, data = None, dist_type = None):
    delta_x = np.array([each['x_coordinate'] for each in data]) - x_test
    delta_y = np.array([each['y_coordinate'] for each in data]) - y_test
    if dist_type == 'Euclidean':
        return np.array(((delta_x**2) + (delta_y**2))**(0.5))
    elif dist_type == 'Manhattan':
        return np.array(np.abs(delta_x) + np.abs(delta_y))

starting_cities = df_dict.copy()
tour = []

starting_idx = np.random.randint(0, len(starting_cities))
tour.append(starting_cities[starting_idx])
starting_cities.pop(starting_idx)

total_distance = 0
for each in range(len(starting_cities)):
    distance_array = distance(x_test = tour[-1]['x_coordinate'], 
                              y_test = tour[-1]['y_coordinate'], 
                              data = starting_cities, 
                              dist_type = 'Euclidean')
    next_closest_city_idx = np.argmin(distance_array)
    total_distance += float(distance_array[next_closest_city_idx])

    tour.append(starting_cities[next_closest_city_idx])
    starting_cities.pop(next_closest_city_idx)
tour.append(tour[0])

results = [{'idx': str(idx), 
            'city': str(each['city']), 
            'state': str(each['state'].strip()), 
            'lat': float(each['lat']), 
            'lon': float(each['lon'])} for idx, each in enumerate(tour)]

location_values = pd.DataFrame(results).values.tolist()

m = folium.Map(location=[39.742043, -104.991531],
               zoom_start=3,
               tiles='OpenStreetMap')

folium.PolyLine([(each['lat'], each['lon']) for each in tour],
                color = 'blue',
                weight = 2,
                opacity = 0.5).add_to(m)

for each in range(0, len(location_values)):
    folium.Marker(location_values[each][3:5],
                  popup='Tour Stop:' + ' ' + str(location_values[each][0]),
                  tooltip = 'Tour Stop:' + ' ' + str(location_values[each][0]) + '<br>'\
                  + 'State: ' + ' ' + str(location_values[each][2]) + '<br>'\
                  + 'City: ' + ' ' + str(location_values[each][1])).add_to(m).save('tsp.html')

print('Starting Location: {}'.format(tour[1]['state']))
print('Total Distance: {:.2f} km'.format(total_distance))
display(m)

Starting Location:  Massachusetts
Total Distance: 21501.79 km
